In [ ]:
url = "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc"

import requests

response = requests.get(url)
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    f.write(response.content)
    print(f"{filename} download done")

zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc download done


In [ ]:
!pip install netCDF4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00


In [ ]:
# open .nc file

import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import sys
import datetime
import scipy.stats as stats

# path: dayavg_zmla_Eday_MIROC-ES2L.nc

# open file
f = nc.Dataset('dayavg_zmla_Eday_MIROC-ES2L.nc', 'r')
ablh = f.variables['zmla'][:]

print(np.shape(ablh))

# get lat and lon length
lat_length = ablh.shape[1]
lon_length = ablh.shape[2]


# robust trend function
def robust_trend(x, y):
    """
    Calculate the slope of a robust regression line.
    """
    return stats.theilslopes(y, x)[0]

# create empty array to store trend values
ablh_trend = np.empty_like(ablh[0,:,:])

# loop through lat and lon indices and estimate trend at each grid point
for i in range(lat_length):
    for j in range(lon_length):
        ablh_grid = ablh[:, i, j]

        # remove missing values
        # ablh_grid = ablh_grid[~np.isnan(ablh_grid)]

        # calculate the trend
        slope = robust_trend(np.arange(ablh.shape[0]), ablh_grid)
        # slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(ablh.shape[0]), ablh[:,i,j])
        ablh_trend[i,j] = slope
        print(i, j, slope)

print(np.shape(ablh_trend))

# plot the trend
plt.imshow(ablh_trend)
plt.colorbar()
plt.show()

